In [1]:
####In this project we will be dealing with designing a recommendation engine
### this recommendation engine will be using content based filtering
### We will be making use of a test file containing the details abount movies
## this test file will be having a feature called overview
## we will be maing recommendation egine on the basis of the overview feature first
### and then we will be building a more advanced recommendation engine by taking into account the features that actually matter
### let's get started

In [3]:
###making the necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
data=pd.read_csv("clean_data.csv")

In [5]:
data.head()

,Unnamed: 0,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,cast,crew
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
#### since we will be building the recommendation engine on the basis of overview features
### let us explore about that feature and general data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

In [10]:
#### we see that there are 4803 entries
###let us explore about the data type of dataset particularly about the overview feature
data.dtypes


Unnamed: 0                int64
budget                    int64
genres                   object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
tagline                  object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
dtype: object

In [11]:
#### we see that overview is a non numerical feature
### Looking at the the data we can see that overview is a string feature
##it has to be converted to a numeric before we feed this to model to make new predictions
##But first let us explore that if this column has some missing values
data['overview'].isna().sum()

3

In [12]:
### we see that that overview column consist of 3 missing values
##let us go ahead and impute these missing values 
data['overview']=data['overview'].fillna('')

In [13]:
###let us verify again if the overview feature has some missing values
data['overview'].isna().sum()

0

In [15]:
###now since we have treated the missing value let us convert this value to number
### for this we will be using TF-IDF technique
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
### we are gonna instatntiate the tfidfVectorizer class and we will be using some parameteres like
### min_df=3 this means that ignore the words that will be appearing in less than 3 documents
## stop_words="english" this means that ignore the stop words that appear in english language
### ngram_range this means that words can be taken in a combination from minimum 1 combination to max 3 combination of words
###max_features=None this means we will have the final matrix as a result of all the features and no features will be ignored
tf=TfidfVectorizer(ngram_range=(1,3),min_df=3,max_features=None,stop_words="english")

In [20]:
###now we can transform the overview feature using the tf instance
tf_matrix=tf.fit_transform(data['overview'])

In [24]:
### we see that we get a sparse matrix with 4803  and 9919 columns
## by sparse matrix we mean that we have few ones and many zeroes
### by 9919 columns we mean we have a total of 9919 words in the document
tf_matrix

<4803x9919 sparse matrix of type '<class 'numpy.float64'>'
	with 121480 stored elements in Compressed Sparse Row format>

In [25]:
###now we have our matrix ready so we are ready to apply cosine similarity to it
## we are doing this so we get a similarity model which will help us to write a method
##this method can help us in recommending movies based on the model we are going to build now
from sklearn.metrics.pairwise import sigmoid_kernel

In [28]:
sgModel=sigmoid_kernel(tf_matrix,tf_matrix)

In [36]:
###we see that we get the similarity scores in each index...
sgModel


array([[0.76163649, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76163649, 0.76159416, ..., 0.76159519, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76163649, ..., 0.76159484, 0.76159416,
        0.76159416],
       ...,
       [0.76159416, 0.76159519, 0.76159484, ..., 0.76163649, 0.76159488,
        0.76159447],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159488, 0.76163649,
        0.76159467],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159447, 0.76159467,
        0.76163649]])

In [31]:
###the above modelcan help us to write a method 
###with the help of this method we can recommend movies to the user
###this method will take the title and model as parameteres as input and will recommend movies to the user
##to implement this method we first need to construct a series object that will have title of movies as index 
## and data of the series will be actually indices of the data frame
### As the title is given , we get to know the index of the movie
###from the index of the moview , we can get to the know the similarity scores using the sigmoid model
### once we have the similarity score , we can sort them and fetch the top 10 records
###we can then fecth the index of the movies have the highest similarity scores
### then we can fetch the names of movies

In [34]:
indexIdentifier=pd.Series(data=data.index,index=data['original_title'])
indexIdentifier

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [72]:
final_recommendations=[]###making an empty list just to store the index for recommended movies
def recommend_movies(title,model):
    index=indexIdentifier[title]###fetched the index of the movie
    sample_list=list(enumerate(list(sgModel[index])))###making a list of recommended movies
    recommenderList=sorted(sample_list,key=lambda x:x[1],reverse=True)###sorting the movies based on the similarity scores
    recommenderList=recommenderList[1:11]###fetching only top 10 movies
    final_recommendations.clear()###clearing the list so that if the fucntion is called twice , results are not appended
    for i in recommenderList:###running the loop to store the index for the recommended movies
        final_recommendations.append(i[0])
    return data.iloc[final_recommendations]['original_title']##returning the title of movies using the index

In [73]:
###testing the recommendation system for the James Bond Movie Spectre
recommend_movies('Spectre',sgModel)

1343          Never Say Never Again
3162                    Thunderball
4071          From Russia with Love
11                Quantum of Solace
3351    The Man with the Golden Gun
1717                     Safe Haven
1200           The Living Daylights
4339                         Dr. No
1281                        Hackers
29                          Skyfall
Name: original_title, dtype: object

In [74]:
##### testing the recommendation system for the Disnep Movie John Carter
recommend_movies('John Carter',sgModel)

1254                          Get Carter
4161         The Marine 4: Moving Target
2932                        Raising Cain
3349                           Desperado
1307                       The Hurricane
91          Independence Day: Resurgence
345                          Rush Hour 2
3068                         Rescue Dawn
16                          The Avengers
4274    Eddie: The Sleepwalking Cannibal
Name: original_title, dtype: object

In [75]:
####Note - The above recommendation engine is designed just using the overview feature